# Рекомендация тарифов

**Описание проекта:**
Даны данные о поведении клиентов, которые уже перешли на тарифы Смарт и Ультра. Нужно построить модель для задачи классификации, которая выберет подходящий тариф с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. 

**Цель исследования - Нужно построить модель для задачи классификации, которая выберет подходящий тариф с максимально большим значением *accuracy* (не менее 0.75)**

**Исследование пройдет в несколько этапов:**
1. Обзор данных;
2. Деление данных на выборки;
3. Исследование моделей;
4. Проверка моделей;
5. Подведение итогов (вывод).

## Откройте и изучите файл

Импортируем необходимые для работы библиотеки.

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Загрузим датасет в переменную и просмотрим данные.

In [3]:
data = pd.read_csv ('/datasets/users_behavior.csv')

In [4]:
data.head(15)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Просмотрим информацию о датасете.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


**Описание данных:**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков. Тип - float64.
* minutes — суммарная длительность звонков в минутах. Тип - float64.
* messages — количество sms-сообщений. Тип - float64.
* mb_used — израсходованный интернет-трафик в Мб. Тип - float64.
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0). Тип - int64.

### Вывод ###

Названия столбцов соотвествует правилам хорошего стиля, пропуски в данных отсутствуют, типы данных соответствуют столбцам.

## Разбейте данные на выборки

Необходимо разделить исходные данные на обучающую, валидационную и тестовую выборки.

In [7]:
data_train, data_valid_test=train_test_split(data, test_size=0.4, random_state=777)
data_valid, data_test=train_test_split(data_valid_test, test_size=0.5, random_state=777)

Проверим успешное разделение на выборки.

In [8]:
print (len(data_train))
print (len(data_valid))
print (len(data_test))

1928
643
643


### Вывод ###

Данные были успешно разделены на обучающую, валидационную и тестовую выборки в соотношении 3:1:1.

## Исследуйте модели

Необходимо исследовать качество разных моделей, меняя гиперпараметры, затем кратко описать выводы исследования.

Выделим признаки и целевой признак.

In [9]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

Посмотрим, как гиперпараметр max_depth дерева решений влияет на качество модели.

In [10]:
best_model_DecisionTreeClassifier = None
best_result_DecisionTreeClassifier = 0
for depth in range(1,20):
    model_DecisionTreeClassifier = DecisionTreeClassifier(random_state=777, max_depth=depth)
    model_DecisionTreeClassifier.fit(features_train, target_train)
    predictions_valid_DecisionTreeClassifier = model_DecisionTreeClassifier.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid_DecisionTreeClassifier)) 
    result_DecisionTreeClassifier = accuracy_score(target_valid, predictions_valid_DecisionTreeClassifier)
    if result_DecisionTreeClassifier > best_result_DecisionTreeClassifier:
        best_model_DecisionTreeClassifier = model_DecisionTreeClassifier
        best_result_DecisionTreeClassifier = result_DecisionTreeClassifier
        best_max_depth = depth
print ('Лучший результат :', best_result_DecisionTreeClassifier, 'max_depth = ', best_max_depth)

max_depth = 1 : 0.7060653188180405
max_depth = 2 : 0.7465007776049767
max_depth = 3 : 0.7698289269051322
max_depth = 4 : 0.7698289269051322
max_depth = 5 : 0.7527216174183515
max_depth = 6 : 0.7620528771384136
max_depth = 7 : 0.7667185069984448
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7667185069984448
max_depth = 10 : 0.7807153965785381
max_depth = 11 : 0.7682737169517885
max_depth = 12 : 0.7573872472783826
max_depth = 13 : 0.7682737169517885
max_depth = 14 : 0.7620528771384136
max_depth = 15 : 0.7387247278382582
max_depth = 16 : 0.7418351477449455
max_depth = 17 : 0.7356143079315708
max_depth = 18 : 0.7402799377916018
max_depth = 19 : 0.7371695178849145
Лучший результат : 0.7807153965785381 max_depth =  10


Посмотрим, как гиперпараметр n_estimators случайного леса влияет на качество модели.

In [11]:
best_model_RandomForestClassifier = None
best_result_RandomForestClassifier = 0
for est in range(1, 51):
    model_RandomForestClassifier = RandomForestClassifier(random_state=777, n_estimators=est) 
    model_RandomForestClassifier.fit(features_train, target_train) 
    predict_RandomForestClassifier = model_RandomForestClassifier.predict(features_valid)
    result_RandomForestClassifier = model_RandomForestClassifier.score(features_valid, target_valid)
    print ("n_estimators =", est, ": ", result_RandomForestClassifier)
    if result_RandomForestClassifier > best_result_RandomForestClassifier:
        best_model_RandomForestClassifier = model_RandomForestClassifier
        best_result_RandomForestClassifier = result_RandomForestClassifier
        best_n_estimators = est
print ('Лучший результат :', best_result_RandomForestClassifier, 'n_estimators = ', best_n_estimators)

n_estimators = 1 :  0.7262830482115086
n_estimators = 2 :  0.7511664074650077
n_estimators = 3 :  0.7418351477449455
n_estimators = 4 :  0.7465007776049767
n_estimators = 5 :  0.744945567651633
n_estimators = 6 :  0.76049766718507
n_estimators = 7 :  0.7527216174183515
n_estimators = 8 :  0.76049766718507
n_estimators = 9 :  0.7620528771384136
n_estimators = 10 :  0.7667185069984448
n_estimators = 11 :  0.76049766718507
n_estimators = 12 :  0.7698289269051322
n_estimators = 13 :  0.7713841368584758
n_estimators = 14 :  0.776049766718507
n_estimators = 15 :  0.7744945567651633
n_estimators = 16 :  0.7791601866251944
n_estimators = 17 :  0.7744945567651633
n_estimators = 18 :  0.776049766718507
n_estimators = 19 :  0.7791601866251944
n_estimators = 20 :  0.7822706065318819
n_estimators = 21 :  0.776049766718507
n_estimators = 22 :  0.7822706065318819
n_estimators = 23 :  0.7776049766718507
n_estimators = 24 :  0.7776049766718507
n_estimators = 25 :  0.7807153965785381
n_estimators = 26 :

Посмотрим на значение качества у логистической регрессии.

In [12]:
model_LogisticRegression = LogisticRegression(random_state=777) 
model_LogisticRegression.fit(features_train,target_train) 
predict_LogisticRegression = model_LogisticRegression.predict(features_valid)
print('accuracy:',accuracy_score(predict_LogisticRegression,target_valid))

accuracy: 0.7262830482115086


### Вывод ###

Наилучшие значения показателя качества у моделей равны:
* Дерева решений - 0.78;
* Случайного леса - 0.79;
* Логистической регрессии - 0.72.

Лучшее значение у модели случайного леса, худшее - у модели логистической регрессии.

## Проверьте модель на тестовой выборке

Необходимо проверить качество модели на тестовой выборке.

Проверим модель случайного леса.

In [13]:
model_forest = RandomForestClassifier(n_estimators=36, random_state=777)
model_forest.fit(features_train, target_train)
prediction_forest=model_forest.predict(features_test)
#print('n_estimators:',estim,end="")
print(' accuracy:',accuracy_score(prediction_forest,target_test))

 accuracy: 0.7962674961119751


### Вывод ###

Лучший и худший показатели качества предсказаний на тестовой выборке аналогичны показателям на валидационной выборке.

## (бонус) Проверьте модели на адекватность

Необходимо сравнить модели со случайной для оценки адекватности моделей.

В теории мы рассматривали пример со случайной моделью, которая дает ответы 0 или 1 с равной вероятностью. Так как наш целевой признак может принимать значения тоже только 0 или 1, для сравнения можно использовать эту модель. Accuracy такой модели - 0,5 (Ответы модели не связаны с правильными ответами, поэтому вероятность угадать 1 равна 50 % — так же и для 0. Accuracy будет равна 0.5).

Каждая из исследованных выше моделей дает более чем на 20% лучшие показатели, что больше качества случайной модели в 1,4 раза (0,7/0,5).

## Итог

Модель для задачи классификации, которая выберет подходящий тариф с максимально большим значением accuracy - случайный лес с гиперпараметрами random_state=777 и n_estimators=36 (0,79). 

Эффективность модели на валидационной выборке подтверждена результатами проверки на тестовой выборке. Максимальное полученное значение accuracy на 40 процентов больше по сравнению с значением accuracy случайной модели.